# CACHED: Context-Aware Chart Element Detection


- Authors: Pengyu Yan, Saleem Ahmed, David Doermann
- Paper: https://arxiv.org/abs/2305.04151
- GitHub: https://github.com/pengyu965/ChartDete

CACHED is an object detection model for charts. It gives bounding boxes for chat elements, title, tick labels, etc. Since I had hard times running this model, I share the compiled packages. This notebook is just about the systems (compile, install and run), and not about machine learning. I didn't train for this competition and all the data are unchanged from authors model.



## LICENSE

- CACHED, addict, terminaltables: MIT
- MMDetection, mmcv: Apache 2.0
- pycocotools: FreeBSD


### NON-COMMERCIAL codes removed from mmcv (Version 4)

Although most of the codes are MIT or Apache, `mvcc` in the dependence contains a little code from NVIDIA StyleGAN. I have removed these NIVIDIA non-commercial codes. Check src/mmcv with `git status` and `git diff`.

https://github.com/open-mmlab/mmcv/blob/main/LICENSES.md

-> https://github.com/open-mmlab/mmcv/blob/2.x/mmcv/ops/csrc/pytorch/cuda/fused_bias_leakyrelu_cuda.cu

-> https://nvlabs.github.io/stylegan2/license.html

```
3.3 Use Limitation. The Work and any derivative works thereof only may be used or intended for use non-commercially. 
```

Unless these codes are removed or replaced, all object detection models in MM detection including CACHED, which depends on mmcv, cannot be used commercially.

## Use compiled packages

In [ ]:
! pip install --no-index --find-links=file:///kaggle/input/cached \
  addict terminaltables pycocotools mmcv_full mmdet

In [ ]:
import glob
from mmdet.apis import init_detector, inference_detector  # depend heavily on mmcv
from IPython.display import Image

In [ ]:
wdir = '/kaggle/input/cached/work_dirs'
config_file = wdir + '/cascade_rcnn_swin-t_fpn_LGF_VCE_PCE_coco_focalsmoothloss/cascade_rcnn_swin-t_fpn_LGF_VCE_PCE_coco_focalsmoothloss.py'
checkpoint_file = wdir + '/cascade_rcnn_swin-t_fpn_LGF_VCE_PCE_coco_focalsmoothloss/checkpoint.pth'

model = init_detector(config_file, checkpoint_file, device='cuda')


di = '/kaggle/input/benetech-making-graphs-accessible'
filenames = glob.glob(di + '/train/images/*.jpg')
filenames.sort()

for filename in filenames:
    result = inference_detector(model, filename)  # list[array]
    break

`result` is a list of arrays for various classes (title, axis labels, ticke labels, ...), and the array has 5 elements for the bounding box coordinates and the confidence in [0, 1].

In [ ]:
model.show_result(filename, result, out_file='./sample_result.jpg')

Image('sample_result.jpg')

##  Create wheel packages

The model is build upon mmcv (https://github.com/open-mmlab/mmcv), and it takes very long time to compile from source. I created the wheel packages and put them in kaggle datasets.

The following steps are not necessary once the wheel packages are created, but useful for different environment or CUDA versions.


### mmcv

https://github.com/open-mmlab/mmcv.git

```
! cp -r /kaggle/input/cached/src .
%cd /kaggle/working/src/mmcv
! CUDA_HOME=/usr/local/cuda-11.3 MMCV_WITH_OPS=1 pip wheel -w /kaggle/working .
```

Be patient and wait for 40 minutes with kaggle notebook. Important to specify the CUDA version same as PyTorch.

### pycocotools

Similarly, download pycocotools and run `pip wheel`.

```
! pip donwload pycocotools  # => tar.gz source code
````

This also contains CUDA codes and convinient to compile to whl file.

### CACHED as mmdet

```
git clone https://github.com/pengyu965/ChartDete
```

This become the mmdet wheel package.


In [ ]:
import torch
torch.version.cuda